In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda


In [2]:
torch.cuda.is_available()

False

In [3]:
training_data = datasets.FashionMNIST(root='data', train= True, download=True, transform= ToTensor() )
test_data = datasets.FashionMNIST(root='data', train= False, download= True, transform= ToTensor())

In [4]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [8]:
model = NeuralNetwork()

In [9]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [10]:
loss_fn = nn.CrossEntropyLoss()

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [12]:
def test_loop(dataloader, model, loss_fn, ):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295581  [    0/60000]
loss: 2.290438  [ 6400/60000]
loss: 2.276539  [12800/60000]
loss: 2.278729  [19200/60000]
loss: 2.273959  [25600/60000]
loss: 2.252196  [32000/60000]
loss: 2.276104  [38400/60000]
loss: 2.251367  [44800/60000]
loss: 2.240798  [51200/60000]
loss: 2.229467  [57600/60000]
Test Error: 
 Accuracy: 34.3%, Avg loss: 0.035049 

Epoch 2
-------------------------------
loss: 2.234874  [    0/60000]
loss: 2.231728  [ 6400/60000]
loss: 2.187141  [12800/60000]
loss: 2.210841  [19200/60000]
loss: 2.191213  [25600/60000]
loss: 2.155420  [32000/60000]
loss: 2.216372  [38400/60000]
loss: 2.162703  [44800/60000]
loss: 2.155878  [51200/60000]
loss: 2.134834  [57600/60000]
Test Error: 
 Accuracy: 37.6%, Avg loss: 0.033558 

Epoch 3
-------------------------------
loss: 2.143616  [    0/60000]
loss: 2.138098  [ 6400/60000]
loss: 2.048549  [12800/60000]
loss: 2.106218  [19200/60000]
loss: 2.059791  [25600/60000]
loss: 2.014601  [32000/600

In [14]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
